## Лабораторная работа 4.2

Реализовать метод стрельбы и конечно-разностный метод решения краевой задачи для ОДУ в виде программ. С использованием разработанного программного обеспечения решить краевую задачу для обыкновенного дифференциального уравнения 2-го порядка на указанном отрезке. Оценить погрешность численного решения с использованием метода Рунге – Ромберга и путем сравнения с точным решением

In [4]:
import numpy as np


def func(x, y, z):
    return (-4 * x * z + 4 * y) / (2 * x + 1)


def answer(x):
    return x + np.exp(-2 * x)


def g(x, y, k):
    return k


def first(x, y, x0):
    i = 0
    while i < len(x) - 1 and x[i + 1] < x0:
        i += 1
    return (y[i + 1] - y[i]) / (x[i + 1] - x[i])


def rungeKutt(f, a, b, h, y0, y_der):
    n = int((b - a) / h)
    x = [i for i in np.arange(a, b + h, h)]
    y = [y0]
    k = [y_der]
    for i in range(n):
        K1 = h * g(x[i], y[i], k[i])
        L1 = h * f(x[i], y[i], k[i])
        K2 = h * g(x[i] + 0.5 * h, y[i] + 0.5 * K1, k[i] + 0.5 * L1)
        L2 = h * f(x[i] + 0.5 * h, y[i] + 0.5 * K1, k[i] + 0.5 * L1)
        K3 = h * g(x[i] + 0.5 * h, y[i] + 0.5 * K2, k[i] + 0.5 * L2)
        L3 = h * f(x[i] + 0.5 * h, y[i] + 0.5 * K2, k[i] + 0.5 * L2)
        K4 = h * g(x[i] + h, y[i] + K3, k[i] + L3)
        L4 = h * f(x[i] + h, y[i] + K3, k[i] + L3)
        y.append(y[i] + (K1 + 2 * K2 + 2 * K3 + K4) / 6)
        k.append(k[i] + (L1 + 2 * L2 + 2 * L3 + L4) / 6)
    return x, y, k


def stop(y, y1, eps):
    if abs(y[-1] - y1) > eps:
        return True
    else:
        return False


def newN(n_last, n, ans_last, ans, b, y1):
    x, y = ans_last[0], ans_last[1]
    phi_last = y[-1] - y1
    x, y = ans[0], ans[1]
    phi = y[-1] - y1
    return n - (n - n_last) / (phi - phi_last) * phi


def shooting(a, b, y0, y1, h, eps):
    n_last = 1
    n = 0.8
    y_der = n_last
    ans_last = rungeKutt(func, a, b, h, n_last, y_der)[:2]
    y_der = n
    ans = rungeKutt(func, a, b, h, n, y_der)[:2]

    while stop(ans[1], y1, eps):
        n, n_last = newN(n_last, n, ans_last, ans, b, y1), n
        ans_last = ans
        y_der = n
        ans = rungeKutt(func, a, b, h, y0, y_der)[:2]

    return ans[1]


def p(x):
    return 4 * x / (2 * x + 1)


def q(x):
    return -4 / (2 * x + 1)


def f_x(x):
    return 0


def tma(a, b, c, d, shape):
    p = [-c[0] / b[0]]
    q = [d[0] / b[0]]
    x = [0] * (shape + 1)
    for i in range(1, shape):
        p.append(-c[i] / (b[i] + a[i] * p[i - 1]))
        q.append((d[i] - a[i] * q[i - 1]) / (b[i] + a[i] * p[i - 1]))
    for i in reversed(range(shape)):
        x[i] = p[i] * x[i + 1] + q[i]
    return x[:-1]


def fdm(a1, b1, alpha_0, beta_0, alpha_1, beta_1, A, B, h, x1):
    x = [a1]
    a, b, c, d = [], [], [], []
    n = len(x1)
    a.append(0)
    b.append(alpha_0 - beta_0 / h)
    c.append(beta_0 / h)
    d.append(A)
    x.append(x[0] + h)
    Lef = np.zeros((n, n))
    Lef[0, 0] = alpha_0 - beta_0 / h
    Lef[0, 1] = beta_0 / h
    for i in range(1, n - 1):
        a.append(1 / (h ** 2) - p(x[i]) / (2 * h))
        b.append(-2 / (h ** 2) + q(x[i]))
        c.append(1 / (h ** 2) + p(x[i]) / (2 * h))
        d.append(f_x(x[i]))
        x.append(round(x[i] + h, 3))
        Lef[i, i - 1] = 1 / (h ** 2) - p(x[i]) / (2 * h)
        Lef[i, i] = -2 / (h ** 2) + q(x[i])
        Lef[i, i + 1] = 1 / (h ** 2) + p(x[i]) / (2 * h)

    a.append(- beta_1 / h)
    b.append(alpha_1 + beta_1 / h)  # ???????
    c.append(0)
    d.append(B)
    Lef[n - 1, n - 2] = - beta_1 / h
    Lef[n - 1, n - 1] = alpha_1 + beta_1 / h

    y = tma(a, b, c, d, len(a))
    D = np.array(d)
    D = D.transpose()
    RES = np.linalg.solve(Lef, D)

    return y


dy0 = 1
dy1 = 3
a = 0.0
b = 1.0
h1 = 0.1
h2 = h1 / 2
alpha0, beta0 = 1, 0
alpha1, beta1 = 2, 1
x1 = np.arange(a, b + h1, h1)
x2 = np.arange(a, b + h2, h2)
y0 = 0.0
y1 = answer(b)

Y = [answer(i) for i in x1]
eps = 0.000001
n1 = 1
n2 = 2

y1_shoot = shooting(a, b, y0, y1, h1, eps)
y2_shoot = shooting(a, b, y0, y1, h2, eps)
y1_krm = fdm(a, b, alpha0, beta0, alpha1, beta1, dy0, dy1, h1, x1)
y2_krm = fdm(a, b, alpha0, beta0, alpha1, beta1, dy0, dy1, h2, x2)

run1 = []
run2 = []

print("========================================================")
print("|  x  |      y      | Метод стрельбы |    Погрешность  |")
print("========================================================")
for k in range(11):
    print(f"| {x1[k]:.1f} | {Y[k]:11.6f} | {y1_shoot[k]:14.6f} |  {abs(Y[k] - y1_shoot[k]):14.6f} |")
print("========================================================")
print()

print("==================================================================")
print("|  x  |      y      | Конечно-разностный метод |    Погрешность  |")
print("==================================================================")
for k in range(11):
    print(f"| {x1[k]:.1f} | {Y[k]:11.6f} | {y1_krm[k]:24f} |  {abs(Y[k] - y1_krm[k]):14.6f} |")
print("==================================================================")
print()

|  x  |      y      | Метод стрельбы |    Погрешность  |
| 0.0 |    1.000000 |       0.000000 |        1.000000 |
| 0.1 |    0.918731 |       0.113534 |        0.805197 |
| 0.2 |    0.870320 |       0.227067 |        0.643253 |
| 0.3 |    0.848812 |       0.340601 |        0.508211 |
| 0.4 |    0.849329 |       0.454134 |        0.395195 |
| 0.5 |    0.867879 |       0.567668 |        0.300212 |
| 0.6 |    0.901194 |       0.681201 |        0.219993 |
| 0.7 |    0.946597 |       0.794735 |        0.151862 |
| 0.8 |    1.001897 |       0.908268 |        0.093628 |
| 0.9 |    1.065299 |       1.021802 |        0.043497 |
| 1.0 |    1.135335 |       1.135335 |        0.000000 |

|  x  |      y      | Конечно-разностный метод |    Погрешность  |
| 0.0 |    1.000000 |                 1.000000 |        0.000000 |
| 0.1 |    0.918731 |                 0.919791 |        0.001060 |
| 0.2 |    0.870320 |                 0.872368 |        0.002048 |
| 0.3 |    0.848812 |                 0.851813 